# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 63 new papers today
          23 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/22 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2208.09503


extracting tarball to tmp_2208.09503...

 done.


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:118: LatexWarning: attempting recovering figure j1036_psf_res_mod_4panel_grey
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))


...

 done.
Retrieving document from  https://arxiv.org/e-print/2208.10426


extracting tarball to tmp_2208.10426... done.
Retrieving document from  https://arxiv.org/e-print/2208.10450


extracting tarball to tmp_2208.10450...

 done.


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Found documentclass in tmp_2208.10450/draft1_AC.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'table_clumps4' from 'tmp_2208.10450/table_clumps4.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'table_clumps3' from 'tmp_2208.10450/table_clumps3.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.09503-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.09503) | **Updated orbital monitoring and dynamical masses for nearby M-dwarf  binaries**  |
|| Per Calissendorff, et al. -- incl., <mark>Samantha Brown-Sevilla</mark>, <mark>Thomas Henning</mark>, <mark>Maud Langlois</mark>, <mark>Michael Meyer</mark> |
|*Appeared on*| *2022-08-23*|
|*Comments*| *Accepted for publication in A&A. 38 pages, 29 figures, 11 tables*|
|**Abstract**| Young M-type binaries are particularly useful for precise isochronal dating by taking advantage of their extended pre-main sequence evolution. Orbital monitoring of these low-mass objects becomes essential in constraining their fundamental properties, as dynamical masses can be extracted from their Keplerian motion. Here, we present the combined efforts of the AstraLux Large Multiplicity Survey, together with a filler sub-programme from the SpHere INfrared Exoplanet (SHINE) project and previously unpublished data from the FastCam lucky imaging camera at the Nordical Optical Telescope (NOT) and the NaCo instrument at the Very Large Telescope (VLT). Building on previous work, we use archival and new astrometric data to constrain orbital parameters for 20 M-type binaries. We identify that eight of the binaries have strong Bayesian probabilities and belong to known young moving groups (YMGs). We provide a first attempt at constraining orbital parameters for 14 of the binaries in our sample, with the remaining six having previously fitted orbits for which we provide additional astrometric data and updated Gaia parallaxes. The substantial orbital information built up here for four of the binaries allows for direct comparison between individual dynamical masses and theoretical masses from stellar evolutionary model isochrones, with an additional three binary systems with tentative individual dynamical mass estimates likely to be improved in the near future. We attained an overall agreement between the dynamical masses and the theoretical masses from the isochrones based on the assumed YMG age of the respective binary pair. The two systems with the best orbital constrains for which we obtained individual dynamical masses, J0728 and J2317, display higher dynamical masses than predicted by evolutionary models. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.09877-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.09877) | **ATOMS: ALMA Three-millimeter Observations of Massive Star-forming  regions -- XII: Fragmentation and multi-scale gas kinematics in protoclusters  G12.42+0.50 and G19.88-0.53**  |
|| Anindya Saha, et al. -- incl., <mark>Chang Won Lee</mark>, <mark>Yong Zhang</mark> |
|*Appeared on*| *2022-08-23*|
|*Comments*| *23 pages, 17 figures, accepted for publication in MNRAS*|
|**Abstract**| We present new continuum and molecular line data from the ALMA Three-millimeter Observations of Massive Star-forming regions (ATOMS) survey for the two protoclusters, G12.42+0.50 and G19.88-0.53. The 3 mm continuum maps reveal seven cores in each of the two globally contracting protoclusters. These cores satisfy the radius-mass relation and the surface mass density criteria for high-mass star formation. Similar to their natal clumps, the virial analysis of the cores suggests that they are undergoing gravitational collapse ($\rm \alpha_{vir} << 2$). The clump to core scale fragmentation is investigated and the derived core masses and separations are found to be consistent with thermal Jeans fragmentation. We detect large-scale filamentary structures with velocity gradients and multiple outflows in both regions. Dendrogram analysis of the H$^{13}$CO$^{+}$ map identifies several branch and leaf structures with sizes $\sim$ 0.1 and 0.03 pc, respectively. The supersonic gas motion displayed by the branch structures is in agreement with the Larson power-law indicating that the gas kinematics at this spatial scale is driven by turbulence. The transition to transonic/subsonic gas motion is seen to occur at spatial scales of $\sim$0.1 pc indicating the dissipation of turbulence. In agreement with this, the leaf structures reveal gas motions that deviate from the slope of Larson's law. From the large-scale converging filaments to the collapsing cores, the gas dynamics in G12.42+0.50 and G19.88-0.53 show scale-dependent dominance of turbulence and gravity and the combination of these two driving mechanisms needs to be invoked to explain massive star formation in the protoclusters. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.10257-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.10257) | **Trapping (sub-)Neptunes similar to TOI-216b at the inner disk rim:  Implications for the disk viscosity and the Neptunian desert**  |
|| Ondřej Chrenko, Raúl Ortega Chametla, David Nesvorný, <mark>Mario Flock</mark> |
|*Appeared on*| *2022-08-23*|
|*Comments*| *Accepted for publication in A&A, 21 pages, 16 figures*|
|**Abstract**| [Abridged] The occurrence rate of observed sub-Neptunes has a break at 0.1 au, which is often attributed to a migration trap at the inner rim of protoplanetary disks where a positive corotation torque prevents inward migration. We argue that conditions in inner disk regions are such that sub-Neptunes are likely to open gaps, deplete their corotation regions, loose the support of the corotation torque and the trapping efficiency then becomes uncertain. We study what it takes to trap such gap-opening planets at the inner disk rim. We perform 2D locally isothermal and non-isothermal hydrodynamic simulations of planet migration. A viscosity transition is introduced in the disk to (i) create a density drop and (ii) mimic the viscosity increase as the planet migrates from a dead zone towards a region with active magneto-rotational instability (MRI). We choose TOI-216b as a Neptune-like upper-limit test case but we also explore different planetary masses, both on fixed and evolving orbits. For planet-to-star mass ratios $q\simeq(4$-$8)\times10^{-5}$, the density drop at the disk rim becomes reshaped due to gap opening and is often replaced with a small density bump centered on the planet's corotation. Trapping is possible only if the bump retains enough gas mass and if the corotation region becomes azimuthally asymmetric, with an island of librating streamlines that accumulate a gas overdensity ahead of the planet. The overdensity exerts a positive torque that can counteract the negative torque of spiral arms. In our model, efficient trapping depends on the $\alpha$-viscosity and its contrast across the viscosity transition. In order to trap TOI-216b, $\alpha_{\mathrm{DZ}}=10^{-3}$ in the dead zone requires $\alpha_{\mathrm{MRI}}\gtrsim5\times10^{-2}$ in the MRI-active zone. If $\alpha_{\mathrm{DZ}}=5\times10^{-4}$, $\alpha_{\mathrm{MRI}}\gtrsim7.5\times10^{-2}$ is needed. |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.10426-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.10426) | **LAEs at Cosmic Dawn: Implications and Predictions**  |
|| Eduard Salvador-Solé, et al. -- incl., <mark>J. Miguel Mas-Hesse</mark> |
|*Appeared on*| *2022-08-23*|
|*Comments*| *16 pages, 10 figures, accepted for publication in The Astrophysical Journal*|
|**Abstract**| The detection of Lyman-$\alpha$ emitting galaxies (LAEs) puts severe constraints on the reionization history. In this Paper we derive the properties of very high-$z$ LAEs predicted in the only two reionization scenarios shown in a previous Paper to be consistent with current data on 15 independent evolving global (or averaged) cosmic properties regarding luminous objects and the intergalactic medium and the optical depth to electron scattering of ionized hydrogen to CMB photons: one with a monotonic behavior, which is completed by $z=6$, as commonly considered, and another one with a non-monotonic behavior with two full ionization events at $z=6$ and $z=10$. We find that the \Lya luminosity functions of very high-$z$ LAEs are very distinct in those two scenarios. Thus, comparing these predictions to the observations that will soon be available thanks to new instruments such as the James Webb Space Telescope, it should be possible to unveil the right reionization scenario. In the meantime, we can compare the predicted redshift distribution and UV (or Lyman-$\alpha$) luminosities of very high-$z$ LAEs to those of the few objects already observed at $z>7.5$. By doing that we find that such data are in tension with the single reionization scenario, while they are fully compatible with the double reionization scenario. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.10450-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.10450) | **Star formation at the smallest scales; A JWST study of the clump  populations in SMACS0723**  |
|| Adélaïde Claeyssens, et al. -- incl., <mark>Guillaume Mahler</mark> |
|*Appeared on*| *2022-08-23*|
|*Comments*| *Submitted to MNRAS, comments and feedback are welcome*|
|**Abstract**| We present the clump populations detected in 18 lensed galaxies at redshifts 1 to 8 within the lensing cluster field SMACS0723. The recent JWST Early Release Observations of this poorly known region of the sky, have revealed numerous point-like sources within and surrounding their host galaxies, undetected in the shallower HST images. We use JWST multiband photometry and the lensing model of this galaxy cluster to estimate the intrinsic sizes and magnitudes of these stellar clumps. We derive optical restframe effective radii from $<$10 to 100s pc and masses ranging from $\sim10^5$ to $10^8$ M$_{\odot}$, overlapping with massive star clusters in the local universe. The ages range from 1 to 100s Myr. We compare the crossing time to the age of the clumps and determine that between 30 and 50 \% of the detected clumps are consistent with being gravitationally bound. The lack of Gyr old clumps suggest that the dissolution time scales are shorter than 1 Gyr. We see a significant increase in the luminosity (mass) surface density of the clumps with redshift. Clumps in galaxies at the reionisation era have stellar densities higher than massive clusters in the local universe. We zoom-in into single galaxies at redshift $<6$ and find for two galaxies, the Sparkler and the Firework, that their star clusters/clumps show distinctive color distributions and location within and surrounding their host galaxy that are compatible with recent (100-500 Myr) merger events. Our study, conducted on a small sample of galaxies, shows the potential of JWST observations for understanding the conditions under which star clusters form in rapidly evolving galaxies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.10351-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.10351) | **TOI-1468: A system of two transiting planets, a super-Earth and a  mini-Neptune, on opposite sides of the radius valley**  |
|| P. Chaturvedi, et al. -- incl., <mark>Th. Henning</mark>, <mark>E. Herrero</mark> |
|*Appeared on*| *2022-08-23*|
|*Comments*| *26 pages, 15 figures*|
|**Abstract**| We report the discovery and characterization of two small transiting planets orbiting the bright M3.0V star TOI-1468 (LSPM J0106+1913), whose transit signals were detected in the photometric time series in three sectors of the TESS mission. We confirm the e planetary nature of both of them using precise radial velocity measurements from the CARMENES and MAROON-X spectrographs, and supplement them with ground-based transit photometry. A joint analysis of all these data reveals that the shorter-period planet, TOI-1468 b ($P_{\rm b}$ = 1.88 d), has a planetary mass of $M_{\rm b} = 3.21\pm0.24$ $M_{\oplus}$ and a radius of $R_{\rm b} =1.280^{+0.038}_{-0.039} R_{\oplus}$, resulting in a density of $\rho_{\rm b} = 8.39^{+ 1.05}_{- 0.92}$ g cm$^{-3}$, which is consistent with a mostly rocky composition. For the outer planet, TOI-1468 c ($P_{\rm c} = 15.53$ d), we derive a mass of $M_{\rm c} = 6.64^{+ 0.67}_{- 0.68}$ $M_{\oplus}$, a radius of $R_{\rm c} = 2.06\pm0.04\,R_{\oplus}$, and a bulk density of $\rho_{c} = 2.00^{+ 0.21}_{- 0.19}$ g cm$^{-3}$, which corresponds to a rocky core composition with a H/He gas envelope. These planets are located on opposite sides of the radius valley, making our system an interesting discovery as there are only a handful of other systems with the same properties. This discovery can further help determine a more precise location of the radius valley for small planets around M dwarfs and, therefore, shed more light on planet formation and evolution scenarios. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure fig/gemini-sensitivity-832.pdf</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.10345-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.10345) | **Complementary astrometry of Cassini Imaging Science Subsystem Images of  Phoebe**  |
|| <mark>Q.F. Zhang</mark>, et al. |
|*Appeared on*| *2022-08-23*|
|*Comments*| *8 Pages, 5 figures, Journal article*|
|**Abstract**| Phoebe is the only major satellite of Saturn with a retrograde orbit. The Cassini Imaging Science Subsystem (ISS) took a lot of Phoebe images between 2004 and 2017, but only a selection of them has been reduced. In this paper, we reduced the remaining ISS images of Phoebe. In the reduction, the Gaia EDR3 catalogue was used to provide the reference stars' positions, and the modified moment was used to measure the centre of image stars and Phoebe. Finally, a total of 834 ISS images of Phoebe have been reduced successfully. Compared with the JPL ephemeris SAT375, Phoebe's positions are consistent. The average residuals in the right ascension and declination are 0.08" and $-0.05$", and the standard deviations of the residuals are about 0.2". In terms of residuals in linear units, the means in the right ascension and declination are about 5 km and $-2$ km, respectively; The standard deviations are about 11 km. Compared with the JPL ephemeris SAT427 and IMCCE ephemeris PH20, our measurements show a strong bias and a large dispersion. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.10232-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.10232) | **Extending empirical constraints on the SZ-mass scaling relation to  higher redshifts via HST weak lensing measurements of nine clusters from the  SPT-SZ survey at $z\gtrsim1$**  |
|| Hannah Zohren, et al. -- incl., <mark>Beatriz Hernández-Martín</mark>, <mark>Henk Hoekstra</mark>, <mark>Florian Kleinebreil</mark>, <mark>Mischa Schirmer</mark> |
|*Appeared on*| *2022-08-23*|
|*Comments*| *23 pages, 10 figures + appendix (9 pages, 9 figures); accepted for publication in A&A*|
|**Abstract**| We present a Hubble Space Telescope (HST) weak gravitational lensing study of nine distant and massive galaxy clusters with redshifts $1.0 \lesssim z \lesssim 1.7$ ($z_\mathrm{median} = 1.4$) and Sunyaev Zel'dovich (SZ) detection significance $\xi > 6.0$ from the South Pole Telescope Sunyaev Zel'dovich (SPT-SZ) survey. We measured weak lensing galaxy shapes in HST/ACS F606W and F814W images and used additional observations from HST/WFC3 in F110W and VLT/FORS2 in $U_\mathrm{HIGH}$ to preferentially select background galaxies at $z\gtrsim 1.8$, achieving a high purity. We combined recent redshift estimates from the CANDELS/3D-HST and HUDF fields to infer an improved estimate of the source redshift distribution. We measured weak lensing masses by fitting the tangential reduced shear profiles with spherical Navarro-Frenk-White (NFW) models. We obtained the largest lensing mass in our sample for the cluster SPT-CLJ2040$-$4451, thereby confirming earlier results that suggest a high lensing mass of this cluster compared to X-ray and SZ mass measurements. Combining our weak lensing mass constraints with results obtained by previous studies for lower redshift clusters, we extended the calibration of the scaling relation between the unbiased SZ detection significance $\zeta$ and the cluster mass for the SPT-SZ survey out to higher redshifts. We found that the mass scale inferred from our highest redshift bin ($1.2 < z < 1.7$) is consistent with an extrapolation of constraints derived from lower redshifts, albeit with large statistical uncertainties. Thus, our results show a similar tendency as found in previous studies, where the cluster mass scale derived from the weak lensing data is lower than the mass scale expected in a Planck $\nu\Lambda$CDM (i.e. $\nu$ $\Lambda$ Cold Dark Matter) cosmology given the SPT-SZ cluster number counts. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error sequence item 1: expected str instance, TexNode found</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2208.09503.md
    + _build/html/tmp_2208.09503/./isochrone_analysis_all_filled.png
    + _build/html/tmp_2208.09503/./j1036_psf_res_mod_4panel_grey.png
    + _build/html/tmp_2208.09503/./J0437_RV.png
    + _build/html/tmp_2208.09503/./J0459_RV.png
    + _build/html/tmp_2208.09503/./J0532_RV.png
    + _build/html/tmp_2208.09503/./J0613_RV.png
    + _build/html/tmp_2208.09503/./J0728_RV.png
    + _build/html/tmp_2208.09503/./J0916_RV.png
    + _build/html/tmp_2208.09503/./J2317_RV.png
exported in  _build/html/2208.09877.md
    + _build/html/tmp_2208.09877/./R1_fig14_1242outflow_composite_v2.png
    + _build/html/tmp_2208.09877/./R1_fig4_1242+1988_hco+h13co_3sigmacont_v1shift.png
    + _build/html/tmp_2208.09877/./R1_fig3_1242+1988_continuum_map_v1.png
exported in  _build/html/2208.10257.md
    + _build/html/tmp_2208.10257/./figs/at_41_42.png
    + _build/html/tmp_2208.10257/./figs/verification.png
    + _build/html/tmp_2208.10257/./figs/torq_static_noniso_q5e-5.p

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\emptynote}[1]{{$
$  \let\thempfn\relax$
$  \footnotetext[0]{#1}$
$}}$
$\newcommand{\noopsort}[2]{#2}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{\emptynote}[1]{{$
$  \let\thempfn\relax$
$  \footnotetext[0]{#1}$
$}}$
$\newcommand{$\noopsort$}[2]{#2}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.3}$</div>



<div id="title">

# Updated orbital monitoring and dynamical masses for nearby M-dwarf binaries

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2208.09503-b31b1b.svg)](https://arxiv.org/abs/2208.09503)<mark>Appeared on: 2022-08-23</mark> - _Accepted for publication in A&A. 38 pages, 29 figures, 11 tables_

</div>
<div id="authors">

Per Calissendorff, et al. -- incl., <mark><mark>Samantha Brown-Sevilla</mark></mark>, <mark><mark>Thomas Henning</mark></mark>, <mark><mark>Maud Langlois</mark></mark>, <mark><mark>Michael Meyer</mark></mark>

</div>
<div id="abstract">

**Abstract:** Young M-type binaries are particularly useful for precise isochronal dating by taking advantage of their extended pre-main sequence evolution. Orbital monitoring of these low-mass objects becomes essential in constraining their fundamental properties, as dynamical masses can be extracted from their Keplerian motion. Here, we present the combined efforts of the AstraLux Large Multiplicity Survey, together with a filler sub-programme from the SpHere INfrared Exoplanet (SHINE) project and previously unpublished data from the FastCam lucky imaging camera at the Nordical Optical Telescope (NOT) and the NaCo instrument at the Very Large Telescope (VLT). Building on previous work, we use archival and new astrometric data to constrain orbital parameters for 20 M-type binaries. We identify that eight of the binaries have strong Bayesian probabilities and belong to known young moving groups (YMGs). We provide a first attempt at constraining orbital parameters for 14 of the binaries in our sample, with the remaining six having previously fitted orbits for which we provide additional astrometric data and updated Gaia parallaxes. The substantial orbital information built up here for four of the binaries allows for direct comparison between individual dynamical masses and theoretical masses from stellar evolutionary model isochrones, with an additional three binary systems with tentative individual dynamical mass estimates likely to be improved in the near future. We attained an overall agreement between the dynamical masses and the theoretical masses from the isochrones based on the assumed YMG age of the respective binary pair. The two systems with the best orbital constrains for which we obtained individual dynamical masses, J0728 and J2317, display higher dynamical masses than predicted by evolutionary models.

</div>

<div id="div_fig1">

<img src="tmp_2208.09503/./isochrone_analysis_all_filled.png" alt="Fig3" width="100%"/>

**Figure 3. -** Mass-magnitude diagram of the individual components for the targets we were able to resolve and derive dynamical masses for. The filled areas display isochrones with ages ranging from 10, 20, 30, 50, 120 to 400 Myrs from the BHAC15 models. Each binary pair is represented by a different set of symbols, with the secondary component being slightly faded out and having dashed lines for the uncertainty. For J1036 the masses and brightnesses are equal and the two components are overplotted on top of eachother. Magnitudes here are shown as absolute magnitudes, which are also listed in Table \ref{tab:mass} together the distances and corresponding theoretical masses for the approximate age-range of the associated YMG. (*fig:isochrones*)

</div>
<div id="div_fig2">

<img src="tmp_2208.09503/./j1036_psf_res_mod_4panel_grey.png" alt="Fig1" width="100%"/>

**Figure 1. -** Astrometric extraction example for the J1036BC binary. The upper plots display the observed AstraLux data for the binary pair and a PSF reference. The lower plots show the constructed model using two brightness-scaled and position-shifted PSF models, and the residual after subtracting the model from the observed data. The plots here have been normalised to the peak value of the observed data, are scaled linearly in the plots. The colour-bar markers indicate the peak and bottom values for the observed data. The residual for the AstraLux is typically of the order of $\pm 15 \%$, whereas with SPHERE the residual is more commonly around $\pm 1 \%$. (*fig:astrometry*)

</div>
<div id="div_fig3">

<img src="tmp_2208.09503/./J0437_RV.png" alt="Fig2.1" width="14%"/><img src="tmp_2208.09503/./J0459_RV.png" alt="Fig2.2" width="14%"/><img src="tmp_2208.09503/./J0532_RV.png" alt="Fig2.3" width="14%"/><img src="tmp_2208.09503/./J0613_RV.png" alt="Fig2.4" width="14%"/><img src="tmp_2208.09503/./J0728_RV.png" alt="Fig2.5" width="14%"/><img src="tmp_2208.09503/./J0916_RV.png" alt="Fig2.6" width="14%"/><img src="tmp_2208.09503/./J2317_RV.png" alt="Fig2.7" width="14%"/>

**Figure 2. -** Radial velocity fits from the data gathered in Table \ref{tab:RV} used to constrain dynamical masses for the binaries.
	The binaries are listed from left to right in the figure as J0437 and J0459 on the first top row; J0532 and J0613 on the second row from the top; J0728 and J0916 on the third row from the top; and J2317 on the bottom row. The shades of grey represent the first, second and third sigma intervals of the values predicted by the fit for each epoch, where sigma is the standard deviation.
	 (*fig:RV*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\tab}{Table~}$
$\newcommand{\fig}{Fig.~}$
$\newcommand{\sect}{Section.~}$
$\newcommand{\tcr}{\textcolor{red}}$
$\newcommand{\sun}{_\odot}$
$\newcommand{\um}{\rm \mu m}$
$\newcommand{\h2}{H_2}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{$\tab$}{Table~}$
$\newcommand{$\fig$}{Fig.~}$
$\newcommand{$\sect$}{Section.~}$
$\newcommand{$\tcr$}{\textcolor{red}}$
$\newcommand{$\sun$}{_\odot}$
$\newcommand{$\um$}{\rm \mu m}$
$\newcommand{$\h$2}{H_2}$</div>



<div id="title">

# ATOMS: ALMA Three-millimeter Observations of Massive Star-forming regions - XII: Fragmentation and multi-scale gas kinematics in protoclusters G12.42+0.50 and G19.88-0.53

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2208.09877-b31b1b.svg)](https://arxiv.org/abs/2208.09877)<mark>Appeared on: 2022-08-23</mark> - _23 pages, 17 figures, accepted for publication in MNRAS_

</div>
<div id="authors">

Anindya Saha, et al. -- incl., <mark><mark>Chang Won Lee</mark></mark>, <mark><mark>Yong Zhang</mark></mark>

</div>
<div id="abstract">

**Abstract:** We present new continuum and molecular line data from the ALMA Three-millimeter Observations of Massive Star-forming regions (ATOMS) survey for the two protoclusters, G12.42+0.50 and G19.88-0.53. The 3 mm continuum maps reveal seven cores in each of the two globally contracting protoclusters. These cores satisfy the radius-mass relation and the surface mass density criteria for high-mass star formation. Similar to their natal clumps, the virial analysis of the cores suggests that they are undergoing gravitational collapse ($\rm \alpha_{vir} << 2$). The clump to core scale fragmentation is investigated and the derived core masses and separations are found to be consistent with thermal Jeans fragmentation.  We detect large-scale filamentary structures with velocity gradients and multiple outflows in both regions.{\it Dendrogram}analysis of the H$^{13}$CO$^{+}$map identifies several branch and leaf structures with sizes$\sim$0.1 and 0.03 pc, respectively. The supersonic gas motion displayed by the branch structures is in agreement with the Larson power-law indicating that the gas kinematics at this spatial scale is driven by turbulence. The transition to transonic/subsonic gas motion is seen to occur at spatial scales of$\sim$0.1 pc indicating the dissipation of turbulence. In agreement with this, the leaf structures reveal gas motions that deviate from the slope of Larson's law. From the large-scale converging filaments to the collapsing cores, the gas dynamics in G12.42+0.50 and G19.88-0.53 show scale-dependent dominance of turbulence and gravity and the combination of these two driving mechanisms needs to be invoked to explain massive star formation in the protoclusters.

</div>

<div id="div_fig1">

<img src="tmp_2208.09877/./R1_fig14_1242outflow_composite_v2.png" alt="Fig8" width="100%"/>

**Figure 8. -** Outflow emission traced by different tracers overlaid on the moment zero map of SiO (2 - 1) for the region associated with G12.42+0.50. Moment zero map for SiO transition obtained by integrating over the full velocity range is shown as colour scale. (a) SiO contours integrated in velocity ranges [0-16.0]$ \rm km s^{-1}$ and [19.8-26.9]$ \rm km s^{-1}$ for blue and red lobes, respectively, are overlaid on the moment zero map of SiO (2 - 1). The contour levels are 3, 5, 7, 11, 16, 22 times $\sigma$($\sigma$ = 0.03 $\rm Jy beam^{-1} km s^{-1}$), and 3, 4, 5, 9, 13, 18 times $\sigma$($\sigma$ = 0.02 $\rm Jy beam^{-1} km s^{-1}$) for blue and red, respectively. SiO emission only concentrates on several positions labelled as 1 - 6. Black circles represent the area over which the spectra is extracted. (b) Outflow emission traced by HCO$^{+}$ are shown as contours. The contours are integrated in velocity ranges [12-15.0]$ \rm km s^{-1}$ and [21.0-29]$ \rm km s^{-1}$ for blue and red lobes, respectively. The contour levels are 3, 4, 5, 7, 11, 14, 17, 20 times $\sigma$($\sigma$ = 0.05 $\rm Jy beam^{-1} km s^{-1}$), and 3, 4, 5, 6, 7, 8 times $\sigma$($\sigma$ = 0.04 $\rm Jy beam^{-1} km s^{-1}$) for blue and red, respectively. In both the panels, the dashed lines indicate the possible outflow directions. The identified cores are shown as filled ellipses. The beam sizes are shown at the lower left in each figure. (*1242_outflow*)

</div>
<div id="div_fig2">

<img src="tmp_2208.09877/./R1_fig4_1242+1988_hco+h13co_3sigmacont_v1shift.png" alt="Fig2" width="100%"/>

**Figure 2. -**  Clump-averaged spectra of the molecular lines (H$^{13}$CO$^{+}$, HCO$^{+}$) detected towards G12.42+0.50 and G19.88-0.53 using combined 12-m + 7-m data are shown in panel (a) and (b), respectively. The spectra are averaged over a region enclosed by the 3 $\sigma$ contour continuum emission for G12.42 and G19.88, respectively. HCO$^+$ spectra shown in blue (solid) are vertically offset by 0.3 $\rm Jy beam^{-1}$ and H$^{13}$CO$^{+}$ spectra shown in red (dashed) are scaled-up by a factor of 3 in both the panels. The vertical dashed line in each panel marks the LSR velocity of 18.3 and 44.0 $ \rm km s^{-1}$ for G12.42 and G19.88, respectively. (*1242+1988_hco+h13co_3sigma*)

</div>
<div id="div_fig3">

<img src="tmp_2208.09877/./R1_fig3_1242+1988_continuum_map_v1.png" alt="Fig1" width="100%"/>

**Figure 1. -** (a) 3 mm continuum map of the region associated with G12.42+0.50 obtained using 12-m array data from ATOMS is shown in colour scale. Core apertures acquired using CASA-\textit{imfit} are shown in black ellipses. The overlaid contours showing the radio emission at 1390 MHz (beam size is 3.0 arcsec $\times$ 2.4 arcsec ) with contour levels at 3, 6, 9, 18, 63, 150 and 172 times $\sigma$($\sigma$$\sim$ 29.7 $\rm \mu Jy beam^{-1}$) are from Issac19. (b) Same as figure (a) but for G19.88-0.53. The overlaid contours showing the radio emission at 1390 MHz (beam size is 4.3 arcsec $\times$ 2.7 arcsec ) with contour levels at 3, 4, 5 and 6 times $\sigma$($\sigma$$\sim$ 45 $\rm \mu Jy beam^{-1}$) are from Issac20. (c) The colour scale shows the 2.7 mm continuum emission of the region associated with G19.88-0.53 obtained using 12-m array data from the ALMA archive. The black ellipses represent the core apertures acquired using CASA-\textit{imfit}. The beam of the continuum maps are indicated at the bottom left in each figure. (*continuum_maps*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

# Trapping (sub-)Neptunes similar to TOI-216b at the inner disk rim: Implications for the disk viscosity and the Neptunian desert

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2208.10257-b31b1b.svg)](https://arxiv.org/abs/2208.10257)<mark>Appeared on: 2022-08-23</mark> - _Accepted for publication in A&A, 21 pages, 16 figures_

</div>
<div id="authors">

Ondřej Chrenko, Raúl Ortega Chametla, David Nesvorný, <mark><mark>Mario Flock</mark></mark>

</div>
<div id="abstract">

**Abstract:** The occurrence rate of observed sub-Neptunes has a break at 0.1 au,   which is often attributed to a migration trap at the inner rim   of protoplanetary disks where a positive corotation torque prevents inward migration.We argue that conditions in inner disk regions are such that sub-Neptunes   are likely to open gaps, deplete their corotation regions, loose the   support of the corotation torque and the trapping efficiency then becomes uncertain.   We study what it takes to trap such gap-opening planets at the inner disk rim.We perform 2D locally isothermal and non-isothermal hydrodynamic simulations of planet migration.   A viscosity transition is introduced in the disk to   (i) create a density drop and (ii) mimic the viscosity increase as the planet migrates   from a dead zone towards a region with active magneto-rotational instability (MRI).   We choose TOI-216b as a Neptune-like upper-limit test case but we also explore different planetary   masses, both on fixed and evolving orbits.For planet-to-star mass ratios$q\simeq(4$--$8)\times10^{-5}$, the density drop   at the disk rim becomes reshaped due to gap opening and is often   replaced with a small density bump centered on the planet's corotation.   Trapping is possible only if the bump retains enough gas mass and if the corotation region becomes azimuthally asymmetric, with an island of librating streamlines that accumulate a gas overdensity ahead of the planet.   The overdensity exerts a positive torque that can counteract the negative torque of spiral arms. Under suitable conditions, the overdensity turns into a Rossby vortex. In our model, efficient trapping   depends on the$\alpha$-viscosity and its contrast across the viscosity transition.   In order to trap TOI-216b,$\alpha_{\mathrm{DZ}}=10^{-3}$in the dead zone requires$\alpha_{\mathrm{MRI}}\gtrsim5\times10^{-2}$in the MRI-active zone. If$\alpha_{\mathrm{DZ}}=5\times10^{-4}$,$\alpha_{\mathrm{MRI}}\gtrsim7.5\times10^{-2}$is needed.We described a new regime of a migration trap relevant for massive (sub-)Neptunes that puts valuable constraints on the levels of turbulent   stress in the inner part of their natal disks.

</div>

<div id="div_fig1">

<img src="tmp_2208.10257/./figs/at_41_42.png" alt="Fig2" width="100%"/>

**Figure 2. -** As in Fig. \ref{fig:at} but for fixed $\alpha_{\mathrm{DZ}}=10^{-3}$
    and different values of the planet-to-star mass ratio $q$(see the plot legend).
    Two cases of the MRI-active viscosity are considered, namely $\alpha_{\mathrm{MRI}}=5\times10^{-2}$(top)
    and $2.5\times10^{-2}$(bottom). Dashed lines correspond to simulations in which the torque is evaluated
    from the whole disk, without any exclusion of the Hill sphere material. Solid lines are affected by the Hill
    cut (see Eq. \ref{eq:hillcut}). For comparison purposes, the case of TOI-216b is labeled and marked with an arrow.
     (*fig:at_q*)

</div>
<div id="div_fig2">

<img src="tmp_2208.10257/./figs/verification.png" alt="Fig7" width="100%"/>

**Figure 7. -** 
    Radial profiles of the surface density $\Sigma$(top, left vertical axis, solid curves),
    $\alpha$ viscosity (top, right vertical axis, dashed curves), temperature $T$(bottom, left vertical axis, solid curves)
    and aspect ratio $h$(bottom, right vertical axis, dashed curves). A comparison is shown between our 2D non-isothermal
    model (black curves) and a 3D vertically resolved radiative hydrostatic model (red curves).
    The bottom panel is split into three regions discussed in Sect. \ref{sec:model_noniso}, namely
    the dust halo (B), the condensation front (C) and the flared disk (D) \citep[e.g.][]{Ueda_etal_2017ApJ...843...49U}.
 (*fig:noniso_disk*)

</div>
<div id="div_fig3">

<img src="tmp_2208.10257/./figs/torq_static_noniso_q5e-5.png" alt="Fig16.1" width="33%"/><img src="tmp_2208.10257/./figs/torq_static_noniso_qTOI.png" alt="Fig16.2" width="33%"/><img src="tmp_2208.10257/./figs/torq_static_noniso_q9e-5.png" alt="Fig16.3" width="33%"/>

**Figure 16. -** Normalized static torque $\gamma\Gamma/\Gamma_{0}$
    measured in our non-isothermal simulations as a function of the planetary
    orbital distance $a_{\mathrm{p}}$. We show measurements for $q=5\times10^{-5}$(left column), $7.3\times10^{-5}$(TOI-216b; middle column)
    and $9\times10^{-5}$(right column). Individual rows correspond to different viscosities $\alpha_{\mathrm{DZ}}$
    in the dead zone (see the boxed labels) and color-coded points with line segments correspond
    to different viscosities $\alpha_{\mathrm{MRI}}$ in the MRI-active zone (see the plot legend).
    The horizontal dashed line is where the total torque is zero.
    We point out that the extent of the vertical axis in the left column differs from the middle
    and right columns.
     (*fig:noniso_statictq*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

23  publications in the last 7 days.
	 _build/html/2208.10257.md
	 _build/html/2208.09877.md
	 _build/html/2208.09503.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers